In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import math
import statistics as s

In [2]:
airdef = pd.read_excel('innovation/01_Innovation 분야 데이터 정의서(통합)/04_Innovation 분야_환경기상데이터(케이웨더)_데이터정의서(행정동추가)_190726.xlsx', sheet_name = '경진대회용 후보 지점')
airdef.drop('Unnamed: 5', axis=1, inplace=True)
airdef.head()

,종로구,스테이션,측정기 등록일,위치,행정동,노원구,스테이션.1,측정기 등록일.1,위치.1,행정동.1
0,31지점,V10O1610252,2017-12-26 00:00:00,교남동 46,교남동,22지점,V10O1610610,#,하계동 280,하계1동
1,NaN,V10O1610546,2018-03-09 00:00:00,세종로 100,"종로 1,2,3,4가동",NaN,V10O1610376,2017-12-27 00:00:00,상계동 734,"상계6,7동"
2,NaN,V10O1610540,2018-03-14 00:00:00,세종로 100,"종로 1,2,3,4가동",NaN,V10O1610293,2017-12-27 00:00:00,상계동 692-2,상계10동
3,NaN,V10O1610542,2018-03-14 00:00:00,세종로 100,"종로 1,2,3,4가동",NaN,V10O1610356,2017-12-27 00:00:00,상계동 1102-17,상계1동
4,NaN,V10O1610543,2018-03-09 00:00:00,세종로 100,"종로 1,2,3,4가동",NaN,V10O1610616,2017-12-23 00:00:00,상계2동 407-39,상계2동


In [ ]:
n_hdong = airdef.loc[:,('스테이션.','행정동.')]
n_hdong.set_index('행정동', inplace=True)
n_hdong

In [ ]:
nowon_st1 = {}

for dong in station1[station1 == 1].index:
    filepath = '/Users/Neun/Desktop/bigcon/innovation/air/jongro/{}.csv'\
    .format(n_hdong.loc[dong]['스테이션'])
    nowon_st1[dong] = pd.read_csv(filepath).drop(['Unnamed: 10','serial','co2','vocs', 'flag', 'humi','noise'],axis=1)
    
    nowon_st1[dong]['pm10'].replace([-999,-9999], np.NaN, inplace=True)
    nowon_st1[dong]['pm25'].replace([-999,-9999], np.NaN, inplace=True)   
    
    nowon_st1[dong]['tm'] = pd.to_datetime(nowon_st1[dong]['tm'], format = '%Y%m%d%H%M')  # tm 변수를 시간 변수로 바꾸고 인덱스로 설정해주기
    temp = [str(i)[:-6] for i in nowon_st1[dong].tm]
    nowon_st1[dong]['temp'] = temp


In [ ]:
미세먼지 = {}

for dong in nowon_st1.keys():
    
    total_list = []  # 미세먼지 시간대별로 묶어준 걸 리스트 안에 넣음
    for temp in nowon_st1[dong]['temp'].unique():
        inside_list = list(nowon_st1[dong][nowon_st1[dong]['temp'] == temp]['pm10'])
        total_list.append([temp, inside_list])

    for i in total_list:
        i[1] = [j for j in i[1] if ~np.isnan(j)]  # na값 날려주고

    미세먼지[dong] = pd.DataFrame(total_list, columns = ['temp', 'mise'])

    gugan = []
    for i, j in 미세먼지[dong].values:
        gugan.append([i, [(k//10) * 10 for k in j]])

    gugan_1 = pd.DataFrame(gugan, columns = ['temp', 'gugan'])


    # 리스트 안에 na값 있으면 mode, median 불가능 
    # -> na값 없앰
    gugan_list = []
    for i in gugan_1.values:
        gugan_list.append([j for j in i[1] if ~np.isnan(j)])

    # 구간값의 mode 구함
    gugan_mode = []
    for i in gugan_list:
        if len(i) == 0:
            gugan_mode.append(np.NaN)
        else:
            gugan_mode.append(max(set(i), key=i.count))

    미세먼지[dong]['gugan_mode'] = gugan_mode
    
    # 미세먼지 중위수 추가
    mise_med = []

    for i in 미세먼지[dong].values:
        if len(i[1]) == 0:
            mise_med.append(np.NaN)
        else:
            mise_med.append(s.median(i[1]))

    미세먼지[dong]['mise_med'] = mise_med

    mise_mode = []
    for i in range(len(미세먼지[dong])):
        if np.isnan(미세먼지[dong]['gugan_mode'][i]):
            mise_mode.append(np.NaN)
        else:
            mise_mode.append(s.mean([j for j in 미세먼지[dong]['mise'][i] if j//10*10 == 미세먼지[dong]['gugan_mode'][i]]))

    미세먼지[dong]['mise_mode'] = mise_mode


In [ ]:
초미세먼지 = {}

for dong in nowon_st1.keys():
    
    total_list = []  # 초미세먼지 시간대별로 묶어준 걸 리스트 안에 넣음
    for temp in nowon_st1[dong]['temp'].unique():
        inside_list = list(nowon_st1[dong][nowon_st1[dong]['temp'] == temp]['pm25'])
        total_list.append([temp, inside_list])

    for i in total_list:
        i[1] = [j for j in i[1] if ~np.isnan(j)]  # na값 날려주고

    초미세먼지[dong] = pd.DataFrame(total_list, columns = ['temp', 'cmise'])

    gugan = []
    for i, j in 초미세먼지[dong].values:
        gugan.append([i, [(k//10) * 10 for k in j]])

    gugan_1 = pd.DataFrame(gugan, columns = ['temp', 'gugan'])


    # 리스트 안에 na값 있으면 mode, median 불가능 
    # -> na값 없앰
    gugan_list = []
    for i in gugan_1.values:
        gugan_list.append([j for j in i[1] if ~np.isnan(j)])

    # 구간값의 mode 구함
    gugan_mode = []
    for i in gugan_list:
        if len(i) == 0:
            gugan_mode.append(np.NaN)
        else:
            gugan_mode.append(max(set(i), key=i.count))

    초미세먼지[dong]['gugan_mode'] = gugan_mode
    
    # 초미세먼지 중위수 추가
    mise_med = []

    for i in 초미세먼지[dong].values:
        if len(i[1]) == 0:
            mise_med.append(np.NaN)
        else:
            mise_med.append(s.median(i[1]))

    초미세먼지[dong]['cmise_med'] = mise_med

    mise_mode = []
    for i in range(len(초미세먼지[dong])):
        if np.isnan(초미세먼지[dong]['gugan_mode'][i]):
            mise_mode.append(np.NaN)
        else:
            mise_mode.append(s.mean([j for j in 초미세먼지[dong]['cmise'][i] if j//10*10 == 초미세먼지[dong]['gugan_mode'][i]]))

    초미세먼지[dong]['cmise_mode'] = mise_mode